<a href="https://colab.research.google.com/github/atoholj/used-car-price-analysis/blob/main/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================
# Used Cars - Exploratory Data Analysis
# ==============================

# --- Upload vehicles.csv ---
from google.colab import files
print("Choose vehicles.csv from your computer…")
uploaded = files.upload()  # pick vehicles.csv

# --- Load dataset ---
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("vehicles.csv", low_memory=False)
print("Original shape:", df.shape)
print(df.head())

# --- Clean columns ---
df.columns = (
    df.columns.str.strip().str.lower()
      .str.replace(" ", "_").str.replace("-", "_")
)

# --- Filter plausible ranges ---
if "price" in df:
    df = df[df["price"].between(500, 200000, inclusive="both")]
if "year" in df:
    df = df[df["year"].between(1985, 2025, inclusive="both")]
if "odometer" in df:
    df = df[df["odometer"].between(0, 400000, inclusive="both")]

print("After filtering:", df.shape)

# --- Feature engineering ---
current_year = datetime.now().year

if "year" in df:
    df["age"] = current_year - df["year"]

if {"odometer","age"}.issubset(df.columns):
    df["miles_per_year"] = df["odometer"] / np.where(df["age"]>0, df["age"], 1)

if "condition" in df:
    df["condition_bucket"] = (
        df["condition"].astype(str).str.lower().map({
            "new":"A_new","like new":"A_new","excellent":"B_excellent",
            "good":"C_good","fair":"D_fair","salvage":"E_salvage"
        }).fillna("Z_unknown")
    )

if "manufacturer" in df:
    premium = {"bmw","audi","mercedes_benz","mercedes-benz","lexus","acura",
               "infiniti","volvo","porsche","tesla","cadillac","lincoln"}
    manuf_norm = df["manufacturer"].astype(str).str.lower().str.replace(" ", "_").str.replace("-", "_")
    df["brand_tier"] = np.where(manuf_norm.isin(premium), "premium", "mainstream")

# --- Summaries ---
print("\nTop manufacturers:")
print(df["manufacturer"].value_counts().head(10))

if "condition_bucket" in df:
    print("\nMedian price by condition:")
    print(df.groupby("condition_bucket")["price"].median().sort_values(ascending=False))

if "brand_tier" in df:
    print("\nMedian price by brand tier:")
    print(df.groupby("brand_tier")["price"].median().sort_values(ascending=False))

# --- Save cleaned dataset ---
clean_path = "vehicles_clean.csv"
df.to_csv(clean_path, index=False)
print("Saved cleaned dataset to:", clean_path, "with shape:", df.shape)


Choose vehicles.csv from your computer…


Saving vehicles.csv to vehicles.csv
Original shape: (426880, 18)
           id                  region  price  year manufacturer model  \
0  7222695916                prescott   6000   NaN          NaN   NaN   
1  7218891961            fayetteville  11900   NaN          NaN   NaN   
2  7221797935            florida keys  21000   NaN          NaN   NaN   
3  7222270760  worcester / central MA   1500   NaN          NaN   NaN   
4  7210384030              greensboro   4900   NaN          NaN   NaN   

  condition cylinders fuel  odometer title_status transmission  VIN drive  \
0       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
1       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
2       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
3       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   
4       NaN       NaN  NaN       NaN          NaN          NaN  NaN   NaN   

  size type paint_color state  
0

In [3]:
from google.colab import files
files.download("vehicles_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>